<a href="https://colab.research.google.com/github/KaranJoseph/DemandForecasting_SCA/blob/main/InventoryManagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Pyomo and GLPK
!pip install -q pyomo
!apt-get install -y -qq glpk-utils #if GLPK is used
# !apt-get install -y -qq coinor-cbc #if cbc is used

     |████████████████████████████████| 9.6 MB 4.8 MB/s 
     |████████████████████████████████| 49 kB 4.2 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 156210 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/KaranJoseph/DemandForecasting_SCA/main/Data/Output.csv")
df = df[["Item_ID", "Pred"]].dropna()

maxx = df["Pred"].max() + 50000 #Big M constraint

In [ ]:
Co = 80000 #Ordering Cost
Ch = 5/12 #Holding Cost

In [ ]:
from pyomo.environ import *

def optimizer(x):
  model = ConcreteModel()
  #Variables 
  model.y = Var(list(range(1, len(x)+1)), within=Binary) # Order yes or no
  model.s = Var(list(range(1, len(x)+1)), within=NonNegativeReals) # Inventory at period i
  model.q = Var(list(range(1, len(x)+1)), within=NonNegativeReals) # Order Quantity at period i

  #Objective
  obj1 = 0
  obj2 = 0
  for i in range(1, len(x)+1):
    obj1 += model.y[i]
    obj2 += model.s[i]
  model.OBJ = Objective(sense=minimize, expr = Co*obj1 + Ch*obj2)

  #Constraints
  model.order1 = Constraint(expr = model.q[1] <= model.y[1]*maxx)
  model.order2 = Constraint(expr = model.q[2] <= model.y[2]*maxx)
  model.order3 = Constraint(expr = model.q[3] <= model.y[3]*maxx)
  model.order4 = Constraint(expr = model.q[4] <= model.y[4]*maxx)
  model.order5 = Constraint(expr = model.q[5] <= model.y[5]*maxx)
  model.order6 = Constraint(expr = model.q[6] <= model.y[6]*maxx)
  model.order7 = Constraint(expr = model.q[7] <= model.y[7]*maxx)
  model.order8 = Constraint(expr = model.q[8] <= model.y[8]*maxx)


  model.inv1 = Constraint(expr = model.s[1] - model.q[1] + x[0] == 0) #Initial Inventory = 0 (Assumption)
  model.inv2 = Constraint(expr = model.s[2] - model.s[1] - model.q[2] + x[1] == 0)
  model.inv3 = Constraint(expr = model.s[3] - model.s[2] - model.q[3] + x[2] == 0)
  model.inv4 = Constraint(expr = model.s[4] - model.s[3] - model.q[4] + x[3] == 0)
  model.inv5 = Constraint(expr = model.s[5] - model.s[4] - model.q[5] + x[4] == 0)
  model.inv6 = Constraint(expr = model.s[6] - model.s[5] - model.q[6] + x[5] == 0)
  model.inv7 = Constraint(expr = model.s[7] - model.s[6] - model.q[7] + x[6] == 0)
  model.inv8 = Constraint(expr = model.s[8] - model.s[7] - model.q[8] + x[7] == 0)

  #model.pprint()
  opt = SolverFactory('glpk')
  opt.solve(model) 
  #model.display()
  return model

In [ ]:
df_im = pd.DataFrame()
cost = {}
for item in df["Item_ID"].unique():
  temp = df[df["Item_ID"] == item].reset_index().drop("index", axis=1)
  model = optimizer(temp["Pred"].values)
  order = []
  qty = []
  inv = []
  for i in range(1, temp.shape[0]+1):
    order.append(model.y[i].value)
    qty.append(model.q[i].value)
    inv.append(model.s[i].value)

  qty = pd.Series(qty, name="OrderQty")
  inv = pd.Series(inv, name="Inventory")
  order = pd.Series(order, name="Order(yes/no)")
  t = pd.concat([temp, qty, inv, order], axis=1)
  df_im = df_im.append(t)

  cost[item] = round(model.OBJ(),2)

In [ ]:
type(t)

pandas.core.frame.DataFrame

In [ ]:
df_im.head()

,Item_ID,Pred,OrderQty,Inventory,Order(yes/no)
0,1267128,13744.703494,5.617225e+04,42427.549007,1.0
1,1267128,13295.465074,0.000000e+00,29132.083933,0.0
2,1267128,15255.420539,7.275958e-12,13876.663394,0.0
3,1267128,13876.663394,0.000000e+00,0.000000,0.0
4,1267128,12821.709087,5.395715e+04,41135.442962,1.0
